#Assignment-3 Implementation
Prepared by Pelinsu Sarac - 28820


##1- Loading Train Set

In [ ]:
from google.colab import drive
import cv2
import os
import numpy as np
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt

drive.mount('/content/drive')

Mounted at /content/drive


Loading the train set images as binary images. Also storing the class labels for each image.

In [ ]:
train_folder_path = '/content/drive/My Drive/Assignment-3/train'

train_file_list = os.listdir(train_folder_path)

train_dict = []

for image_file in train_file_list:
  image_path = os.path.join(train_folder_path, image_file)
  img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
  img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]
  class_name = image_file.split('-')[0]

  train_dict.append([image_file, img, class_name, None])

  print("Loaded from train:", image_file)


Loaded from train: spring-3.png
Loaded from train: frog-7.png
Loaded from train: tree-5.png
Loaded from train: bottle-10.png
Loaded from train: device0-2.png
Loaded from train: shoe-2.png
Loaded from train: device6-9.png
Loaded from train: device1-6.png
Loaded from train: cellular_phone-3.png
Loaded from train: dog-3.png
Loaded from train: cellular_phone-5.png
Loaded from train: camel-4.png
Loaded from train: device9-7.png
Loaded from train: Misk-3.png
Loaded from train: fly-7.png
Loaded from train: device1-7.png
Loaded from train: horseshoe-6.png
Loaded from train: elephant-3.png
Loaded from train: device7-6.png
Loaded from train: hammer-4.png
Loaded from train: fork-4.png
Loaded from train: apple-9.png
Loaded from train: device7-10.png
Loaded from train: Glas-8.png
Loaded from train: device5-9.png
Loaded from train: fish-2.png
Loaded from train: rat-6.png
Loaded from train: chopper-4.png
Loaded from train: bird-3.png
Loaded from train: pencil-8.png
Loaded from train: ray-10.png
Loade

Similar loading process for test images.

In [ ]:
test_folder_path = '/content/drive/My Drive/Assignment-3/test'

test_file_list = os.listdir(test_folder_path)

test_dict = []

for image_file in test_file_list:
  image_path = os.path.join(test_folder_path, image_file)
  img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
  class_name = image_file.split('-')[0]

  test_dict.append([image_file, img, class_name, None])

  print("Loaded from test:", image_file)

Loaded from test: watch-16.png
Loaded from test: teddy-16.png
Loaded from test: cattle-14.png
Loaded from test: camel-17.png
Loaded from test: jar-14.png
Loaded from test: device4-16.png
Loaded from test: butterfly-18.png
Loaded from test: children-18.png
Loaded from test: HCircle-16.png
Loaded from test: horseshoe-18.png
Loaded from test: chopper-15.png
Loaded from test: fork-11.png
Loaded from test: fountain-15.png
Loaded from test: cellular_phone-17.png
Loaded from test: frog-11.png
Loaded from test: device9-11.png
Loaded from test: cellular_phone-11.png
Loaded from test: ray-18.png
Loaded from test: cattle-13.png
Loaded from test: horseshoe-19.png
Loaded from test: lmfish-13.png
Loaded from test: car-11.png
Loaded from test: brick-20.png
Loaded from test: pocket-11.png
Loaded from test: horse-17.png
Loaded from test: brick-18.png
Loaded from test: carriage-15.png
Loaded from test: shoe-20.png
Loaded from test: children-19.png
Loaded from test: horse-19.png
Loaded from test: device4

Trial image to test whether the descriptors are working.

In [ ]:
trial_image = train_dict[0][1]

print(train_dict[0][0])

spring-3.png


##2- Shape Descriptors

###Basic Shape Descriptors



Basic shape descriptors are obtained via regionprops function from skimage library. Needed descriptors are stored inside a dictionary to be combined later on.


In [ ]:
from skimage.measure import regionprops

def basic_shape_descriptors(img):

  descriptor_dict = {}

  props = regionprops(img.astype(int))[0]

  total_num_pixel = img.shape[0] * img.shape[1]

  descriptor_dict['area'] = props.area
  descriptor_dict['perimeter'] = props.perimeter
  descriptor_dict['convexity'] = props.area / props.convex_area
  descriptor_dict['circularity'] = pow(props.perimeter, 2) / props.area

  min_row, min_col, max_row, max_col = props.bbox
  bbox_area = (max_row - min_row) * (max_col - min_col)
  descriptor_dict['rectangularity'] = props.area / bbox_area
  descriptor_dict['eccentricity'] = props.eccentricity

  return descriptor_dict

In [ ]:
basic_shape_descriptors(trial_image)

{'area': 3049,
 'perimeter': 920.4894654563448,
 'convexity': 0.34498755374519124,
 'circularity': 277.8946723568735,
 'rectangularity': 0.2006317036257156,
 'eccentricity': 0.9635256390210364}

Combinations with size from 3 up to 6.

In [ ]:
from itertools import combinations

all_prop = ['area', 'perimeter', 'convexity', 'circularity', 'rectangularity', 'eccentricity']

basic_combos = []

for r in range(3, 7):

  combo = combinations(all_prop, r)
  basic_combos.extend(combo)

print(basic_combos)

[('area', 'perimeter', 'convexity'), ('area', 'perimeter', 'circularity'), ('area', 'perimeter', 'rectangularity'), ('area', 'perimeter', 'eccentricity'), ('area', 'convexity', 'circularity'), ('area', 'convexity', 'rectangularity'), ('area', 'convexity', 'eccentricity'), ('area', 'circularity', 'rectangularity'), ('area', 'circularity', 'eccentricity'), ('area', 'rectangularity', 'eccentricity'), ('perimeter', 'convexity', 'circularity'), ('perimeter', 'convexity', 'rectangularity'), ('perimeter', 'convexity', 'eccentricity'), ('perimeter', 'circularity', 'rectangularity'), ('perimeter', 'circularity', 'eccentricity'), ('perimeter', 'rectangularity', 'eccentricity'), ('convexity', 'circularity', 'rectangularity'), ('convexity', 'circularity', 'eccentricity'), ('convexity', 'rectangularity', 'eccentricity'), ('circularity', 'rectangularity', 'eccentricity'), ('area', 'perimeter', 'convexity', 'circularity'), ('area', 'perimeter', 'convexity', 'rectangularity'), ('area', 'perimeter', 'c

In [ ]:
len(basic_combos)

31

###Fourier Descriptor

In [ ]:
def Fourier_Descriptors(img, coeffs=-1):

  # Find the border of the object
  contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  largest_contour = max(contours, key=cv2.contourArea)

  # Convert the coordinates of the points on the border to complex numbers
  contour_complex = [point[0][0] + 1j * point[0][1] for point in largest_contour]

  # Obtain the Fourier descriptors by giving the complex number to FFT
  fourier_transform = np.fft.fft(contour_complex)
  real_descriptors = np.abs(fourier_transform) # Absolute for calculating the distance

  if coeffs == -1:
    return real_descriptors
  else:
    return real_descriptors[0:coeffs+1]

In [ ]:
Fourier_Descriptors(trial_image)

array([4.39557477e+04, 1.44310926e+04, 2.64793132e+03, 3.10938625e+03,
       2.23552340e+03, 6.81716142e+02, 2.18621914e+03, 4.44997659e+02,
       1.62584264e+03, 2.79377360e+02, 1.07065417e+03, 1.91297130e+02,
       7.25953028e+02, 4.13078070e+02, 4.32151891e+02, 3.96614831e+02,
       3.52110680e+02, 1.41468059e+02, 2.19078293e+02, 3.40065264e+02,
       1.42104612e+02, 2.25350712e+02, 1.77989711e+02, 1.20157001e+02,
       2.74513208e+02, 1.03134889e+02, 1.13133190e+02, 1.53515769e+02,
       1.73408100e+02, 8.84834526e+01, 1.28754675e+02, 2.30143034e+01,
       5.75688883e+01, 1.27880677e+02, 1.47986320e+02, 6.57076036e+01,
       6.89760333e+01, 4.35477573e+01, 8.45046690e+01, 2.46490403e+01,
       2.48403102e+01, 6.08168905e+01, 7.18494305e+01, 5.29930992e+01,
       2.03844581e+01, 4.58979234e+01, 4.10296444e+01, 4.86230278e+01,
       1.12652790e+01, 1.00090672e+02, 7.05983201e+01, 5.52903054e+01,
       4.52886380e+01, 3.69510486e+01, 6.89450873e+01, 3.86714910e+01,
      

###Shape Histograms

In [ ]:
%pip install mahotas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 19.8 MB/s eta 0:00:00


In [ ]:
from mahotas import center_of_mass

def Shape_Histogram(img, n):

  # Calculate the center of mass
  center = center_of_mass(img)
  center_x = int(center[0])
  center_y = int(center[1])

  # To find the farthest point on image to the center, we just need to check the borders
  contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  largest_contour = max(contours, key=cv2.contourArea)

  distances = []

  # Calculate distance for points on the border from the center
  for point in largest_contour:
    x = point[0][0]
    y = point[0][1]

    distance = np.sqrt((center_x - x)**2 + (center_y - y)**2)
    distances.append(distance)

  # Find the farthest point
  furthest_point_idx = np.argmax(distances)
  max_distance = distances[furthest_point_idx]

  # Farthest distance is equally divided into the number of bins
  # These smaller distances will be used to construct the circles
  radii = np.linspace(0, max_distance, n+1)

  hist_values = []

  for i in range(1, n+1):

    # Two values that define the current band on image
    inner = radii[i-1]
    outer = radii[i]

    # Creating the mask in the form of the band
    # It looks like a bagel, or rather simit :D
    bagel = np.zeros(img.shape)

    bagel = cv2.circle(bagel, (center_x, center_y), int(outer), 1, thickness=-1)
    bagel = cv2.circle(bagel, (center_x, center_y), int(inner), 0, thickness=-1)

    # Apply the mask onto the image
    masked = img*bagel

    # Count the pixels remaining after the mask, these will be the corresponding histogram values
    surface = np.count_nonzero(masked)
    hist_values.append(surface)

  total_surface =  np.sum(img > 1)

  # Normalize the values with total surface to have scale invariance
  return np.array(hist_values) / total_surface

In [ ]:
Shape_Histogram(trial_image, 5)

array([0.16005248, 0.29157101, 0.06165956, 0.21285667, 0.27287635])

###Moment Invariants

Hu moments calculated with the help of OpenCV. However, it only provided the first 7 moments, thus the 8 was calculated separately and added.

In [ ]:
def Hu_Moments(img):

  moments = cv2.moments(img)
  hu_moments = cv2.HuMoments(moments)
  hu_moments.flatten()

  invar_8 = moments['nu11']*((moments['nu30'] + moments['nu12'])**2 - (moments['nu03'] + moments['nu21'])**2) - (moments['nu20'] - moments['nu02'])*(moments['nu30'] + moments['nu12'])*(moments['nu03'] + moments['nu21'])
  hu_moments = np.append(hu_moments, invar_8)

  return hu_moments

In [ ]:
Hu_Moments(trial_image)

array([4.58257011e-03, 1.57612646e-05, 2.66737567e-09, 1.60167938e-09,
       3.29254700e-18, 5.92865975e-12, 3.45172292e-19, 1.14941034e-12])

Combinations of moments with size from 6 to 8.

In [ ]:
from itertools import combinations

invariants = [i for i in range(8)]

invar_combos = []

for r in range(6, 9):

  combo = combinations(invariants, r)
  invar_combos.extend(combo)

print(invar_combos)

[(0, 1, 2, 3, 4, 5), (0, 1, 2, 3, 4, 6), (0, 1, 2, 3, 4, 7), (0, 1, 2, 3, 5, 6), (0, 1, 2, 3, 5, 7), (0, 1, 2, 3, 6, 7), (0, 1, 2, 4, 5, 6), (0, 1, 2, 4, 5, 7), (0, 1, 2, 4, 6, 7), (0, 1, 2, 5, 6, 7), (0, 1, 3, 4, 5, 6), (0, 1, 3, 4, 5, 7), (0, 1, 3, 4, 6, 7), (0, 1, 3, 5, 6, 7), (0, 1, 4, 5, 6, 7), (0, 2, 3, 4, 5, 6), (0, 2, 3, 4, 5, 7), (0, 2, 3, 4, 6, 7), (0, 2, 3, 5, 6, 7), (0, 2, 4, 5, 6, 7), (0, 3, 4, 5, 6, 7), (1, 2, 3, 4, 5, 6), (1, 2, 3, 4, 5, 7), (1, 2, 3, 4, 6, 7), (1, 2, 3, 5, 6, 7), (1, 2, 4, 5, 6, 7), (1, 3, 4, 5, 6, 7), (2, 3, 4, 5, 6, 7), (0, 1, 2, 3, 4, 5, 6), (0, 1, 2, 3, 4, 5, 7), (0, 1, 2, 3, 4, 6, 7), (0, 1, 2, 3, 5, 6, 7), (0, 1, 2, 4, 5, 6, 7), (0, 1, 3, 4, 5, 6, 7), (0, 2, 3, 4, 5, 6, 7), (1, 2, 3, 4, 5, 6, 7), (0, 1, 2, 3, 4, 5, 6, 7)]


In [ ]:
len(invar_combos)

37

##3- Distance Functions

Test values to see whether the functions are working.

In [ ]:
from random import randint
from scipy.spatial.distance import euclidean, cityblock, mahalanobis

train = np.array([randint(0, 100) for i in range(10)])
test = np.array([randint(0, 100) for i in range(10)])

print(train)
print(test)

[ 88  60  15  84  48  45  92  63  87 100]
[50 65 67 31 23 38 18 11 70 82]


###Euclidean Distance

In [ ]:
def Euclidean_Distance(test_img, train_img):
  return euclidean(test_img, train_img)

In [ ]:
Euclidean_Distance(test, train)

128.25365491868058

###Manhattan Distance

In [ ]:
def Manhattan_Distance(test_img, train_img):
  return cityblock(test_img, train_img)

In [ ]:
Manhattan_Distance(test, train)

341

###Chi-squared Distance

In [ ]:
def Chi_Squared_Distance(test_img, train_img):
  return np.sum((test_img - train_img) ** 2 / (test_img + train_img))

In [ ]:
Chi_Squared_Distance(test, train)

167.4019860095416

###Mahalonobis Distance

In [ ]:
from scipy.spatial import distance

def Mahalanobis_Distance(test_img, train_img, regularization_param=1e-5):

  # Calculating the inverse covariance matrix
  data_matrix = np.vstack((test_img, train_img))
  cov_matrix = np.cov(data_matrix, rowvar=False) + regularization_param * np.identity(len(test_img))

  try:
    inv_cov_matrix = np.linalg.inv(cov_matrix)
  except np.linalg.LinAlgError: # Singular matrix errors occured for some samples
    return np.inf

  dist = distance.mahalanobis(test_img, train_img, inv_cov_matrix)

  return dist

In [ ]:
Mahalanobis_Distance(test, train)

1.4142136172792423

##4- Nearest Neighbor

Before this function, feature vectors are calculated for train and test sets. This function finds the smallest distance between the images of train and test sets and then labels the test images accordingly. It uses 4 distance functions defined above.

In [ ]:
def Nearest_Neighbor(train_set, test_set):

  # Accuracy scores
  euclidean_score = 0
  manhattan_score = 0
  chi_sq_score = 0
  mahalanobis_score = 0

  test_len = len(test_set)

  # For each image in test set
  for img in test_set:

    test_feat = img[3]
    real_label = img[2]

    # Winner for each distance function means that the label of feature vector with the smallest distance according to the function
    euclidean_winner = [None, np.inf]
    manhattan_winner = [None, np.inf]
    chi_sq_winner = [None, np.inf]
    mahalanobis_winner = [None, np.inf]

    # Compare the image with all samples in train set
    for sample in train_set:

      train_feat = sample[3]
      train_label = sample[2]

      # If the size of feature vectors are not equal, truncate the longer one into the size of shorter
      if train_feat.shape[0] != test_feat.shape[0]:
        train_feat = train_feat[0:min(train_feat.shape[0], test_feat.shape[0])].copy()
        test_feat = test_feat[0:min(train_feat.shape[0], test_feat.shape[0])].copy()

      # Calculate the distances
      euclidean_dist = Euclidean_Distance(test_feat, train_feat)
      manhattan_dist = Manhattan_Distance(test_feat, train_feat)
      chi_sq_dist = Chi_Squared_Distance(test_feat, train_feat)
      mahalanobis_dist = Mahalanobis_Distance(test_feat, train_feat)

      # Update the winner for each distance function according to the results
      if euclidean_dist < euclidean_winner[1]:
        euclidean_winner[0] = train_label
        euclidean_winner[1] = euclidean_dist

      if manhattan_dist < manhattan_winner[1]:
        manhattan_winner[0] = train_label
        manhattan_winner[1] = manhattan_dist

      if chi_sq_dist < chi_sq_winner[1]:
        chi_sq_winner[0] = train_label
        chi_sq_winner[1] = chi_sq_dist

      if mahalanobis_dist < mahalanobis_winner[1]:
        mahalanobis_winner[0] = train_label
        mahalanobis_winner[1] = mahalanobis_dist

    # Update the scores of distance functions if the winner of them matches with the true label
    if euclidean_winner[0] == real_label:
      euclidean_score += 1

    if manhattan_winner[0] == real_label:
      manhattan_score += 1

    if chi_sq_winner[0] == real_label:
      chi_sq_score += 1

    if mahalanobis_winner[0] == real_label:
      mahalanobis_score += 1

  # Divide each score with total number of test images to have the results as ratio
  print("Euclidean Score:", euclidean_score / test_len)
  print("Manhattan Score:", manhattan_score / test_len)
  print("Chi Squared Score:", chi_sq_score / test_len)
  print("Mahalanobis Score:", mahalanobis_score / test_len)

##5- Experiments

###Basic Descriptors

Forming the feature vectors both for train and test according to the combinations decided beforehand, and run the Nearest Neighbor

In [ ]:
for combo in basic_combos:

  print("Combination to be tested:", combo)

  for train_img in train_dict:

    desc = basic_shape_descriptors(train_img[1])
    vec = np.array([desc[el] for el in combo])
    train_img[3] = vec.copy()

  for test_img in test_dict:

    desc = basic_shape_descriptors(test_img[1])
    vec = np.array([desc[el] for el in combo])
    test_img[3] = vec.copy()

  Nearest_Neighbor(train_dict, test_dict)
  print("----------------------------------------------------------------------")

Combination to be tested: ('area', 'perimeter', 'convexity')
Euclidean Score: 0.2914285714285714
Manhattan Score: 0.2957142857142857
Chi Squared Score: 0.34
Mahalanobis Score: 0.037142857142857144
----------------------------------------------------------------------
Combination to be tested: ('area', 'perimeter', 'circularity')
Euclidean Score: 0.2914285714285714
Manhattan Score: 0.2985714285714286
Chi Squared Score: 0.3442857142857143
Mahalanobis Score: 0.041428571428571426
----------------------------------------------------------------------
Combination to be tested: ('area', 'perimeter', 'rectangularity')
Euclidean Score: 0.2914285714285714
Manhattan Score: 0.2957142857142857
Chi Squared Score: 0.3414285714285714
Mahalanobis Score: 0.03857142857142857
----------------------------------------------------------------------
Combination to be tested: ('area', 'perimeter', 'eccentricity')
Euclidean Score: 0.2914285714285714
Manhattan Score: 0.2957142857142857
Chi Squared Score: 0.33571

###Moment Invariants

Forming the feature vectors both for train and test according to the combinations decided beforehand, and run the Nearest Neighbor

In [ ]:
for combo in invar_combos:

  print("Combination to be tested:", combo)

  for train_img in train_dict:

    desc = Hu_Moments(train_img[1])
    vec = np.array([desc[el] for el in combo])
    train_img[3] = vec.copy()

  for test_img in test_dict:

    desc = Hu_Moments(test_img[1])
    vec = np.array([desc[el] for el in combo])
    test_img[3] = vec.copy()

  Nearest_Neighbor(train_dict, test_dict)
  print("----------------------------------------------------------------------")

Combination to be tested: (0, 1, 2, 3, 4, 5)
Euclidean Score: 0.15857142857142856
Manhattan Score: 0.16142857142857142
Chi Squared Score: 0.2985714285714286
Mahalanobis Score: 0.15857142857142856
----------------------------------------------------------------------
Combination to be tested: (0, 1, 2, 3, 4, 6)


<ipython-input-15-b76ca093f216>:2: RuntimeWarning: divide by zero encountered in divide
  return np.sum((test_img - train_img) ** 2 / (test_img + train_img))


Euclidean Score: 0.15857142857142856
Manhattan Score: 0.16142857142857142
Chi Squared Score: 0.2957142857142857
Mahalanobis Score: 0.15857142857142856
----------------------------------------------------------------------
Combination to be tested: (0, 1, 2, 3, 4, 7)
Euclidean Score: 0.15857142857142856
Manhattan Score: 0.16142857142857142
Chi Squared Score: 0.29285714285714287
Mahalanobis Score: 0.15857142857142856
----------------------------------------------------------------------
Combination to be tested: (0, 1, 2, 3, 5, 6)
Euclidean Score: 0.15857142857142856
Manhattan Score: 0.16142857142857142
Chi Squared Score: 0.2957142857142857
Mahalanobis Score: 0.15857142857142856
----------------------------------------------------------------------
Combination to be tested: (0, 1, 2, 3, 5, 7)
Euclidean Score: 0.15857142857142856
Manhattan Score: 0.16142857142857142
Chi Squared Score: 0.29285714285714287
Mahalanobis Score: 0.15857142857142856
----------------------------------------------

###Shape Histogram

Forming the feature vectors both for train and test according to the bin numbers decided, and run the Nearest Neighbor

In [ ]:
for bin_num in [3, 5, 7, 11]:

  print("Bin number to be tested:", bin_num)

  for train_img in train_dict:

    train_img[3] = Shape_Histogram(train_img[1], bin_num)

  for test_img in test_dict:

    test_img[3] = Shape_Histogram(test_img[1], bin_num)

  Nearest_Neighbor(train_dict, test_dict)
  print("----------------------------------------------------------------------")

Bin number to be tested: 3


<ipython-input-15-b76ca093f216>:2: RuntimeWarning: invalid value encountered in divide
  return np.sum((test_img - train_img) ** 2 / (test_img + train_img))


Euclidean Score: 0.22714285714285715
Manhattan Score: 0.23
Chi Squared Score: 0.2
Mahalanobis Score: 0.22714285714285715
----------------------------------------------------------------------
Bin number to be tested: 5
Euclidean Score: 0.38571428571428573
Manhattan Score: 0.3842857142857143
Chi Squared Score: 0.3242857142857143
Mahalanobis Score: 0.38571428571428573
----------------------------------------------------------------------
Bin number to be tested: 7
Euclidean Score: 0.4357142857142857
Manhattan Score: 0.44571428571428573
Chi Squared Score: 0.3585714285714286
Mahalanobis Score: 0.4357142857142857
----------------------------------------------------------------------
Bin number to be tested: 11
Euclidean Score: 0.4642857142857143
Manhattan Score: 0.47714285714285715
Chi Squared Score: 0.3457142857142857
Mahalanobis Score: 0.4642857142857143
----------------------------------------------------------------------


###Fourier Descriptors

Checking the maximum and minimum number of descriptors, along with the median to decide the coefficient values to be decided.

In [ ]:
train_len = []
test_len = []

for train_img in train_dict:

  train_img[3] = Fourier_Descriptors(train_img[1])
  train_len.append(len(train_img[3]))

for test_img in test_dict:

  test_img[3] = Fourier_Descriptors(test_img[1])
  test_len.append(len(test_img[3]))

print("Minimum length for train descriptors:", min(train_len))
print("Maximum length for train descriptors:", max(train_len))
print("Median for train descriptors:", np.median(train_len))

print("Minimum length for test descriptors:", min(test_len))
print("Maximum length for test descriptors:", max(test_len))
print("Median for test descriptors:", np.median(test_len))

Minimum length for train descriptors: 4
Maximum length for train descriptors: 3774
Median for train descriptors: 469.5
Minimum length for test descriptors: 30
Maximum length for test descriptors: 3294
Median for test descriptors: 438.0


New Mahalanobis function is defined due to the Singular Matrix issues. Only difference is that, instead of inverse of the covariance matrix, pseudo-inverse is used.

In [ ]:
from scipy.spatial import distance

def Mahalanobis_Distance(test_img, train_img, regularization_param=1e-5):

  data_matrix = np.vstack((test_img, train_img))
  cov_matrix = np.cov(data_matrix, rowvar=False) + regularization_param * np.identity(len(test_img))

  inv_cov_matrix = np.linalg.pinv(cov_matrix)


  dist = distance.mahalanobis(test_img, train_img, inv_cov_matrix)

  return dist

Coefficient counts are decided according to the medians.

In [ ]:
coeff_count = [50, 100, 200, 400]

for count in coeff_count:

  print("Descriptor count to be tested:", count)

  for train_img in train_dict:

    train_img[3] = Fourier_Descriptors(train_img[1], count)

  for test_img in test_dict:

    test_img[3] = Fourier_Descriptors(test_img[1], count)

  Nearest_Neighbor(train_dict, test_dict)
  print("----------------------------------------------------------------------")

Descriptor count to be tested: 50


<ipython-input-17-b76ca093f216>:2: RuntimeWarning: invalid value encountered in divide
  return np.sum((test_img - train_img) ** 2 / (test_img + train_img))


Euclidean Score: 0.3357142857142857
Manhattan Score: 0.2842857142857143
Chi Squared Score: 0.3842857142857143
Mahalanobis Score: 0.011428571428571429
----------------------------------------------------------------------
Descriptor count to be tested: 100
Euclidean Score: 0.3357142857142857
Manhattan Score: 0.28285714285714286
Chi Squared Score: 0.3842857142857143
Mahalanobis Score: 0.004285714285714286
----------------------------------------------------------------------
Descriptor count to be tested: 200
Euclidean Score: 0.3357142857142857
Manhattan Score: 0.28285714285714286
Chi Squared Score: 0.3842857142857143
Mahalanobis Score: 0.005714285714285714
----------------------------------------------------------------------
Descriptor count to be tested: 400
Euclidean Score: 0.3357142857142857
Manhattan Score: 0.28285714285714286
Chi Squared Score: 0.3842857142857143
Mahalanobis Score: 0.005714285714285714
----------------------------------------------------------------------


###Arbitrary Combinations

Arbitrary combinations are created by randomly selecting tested values for each descriptor. 5 combinations are created.

In [ ]:
from random import randint

bin_nums = [3, 5, 7, 11]
coeff_count = [50, 100, 200, 400]
rand_comb = []

for i in range(5):

  comb = []
  comb.append(basic_combos[randint(0, 30)])
  comb.append(invar_combos[randint(0, 36)])
  comb.append(bin_nums[randint(0,3)])
  comb.append(coeff_count[randint(0,3)])

  print(comb)
  rand_comb.append(comb)

[('perimeter', 'convexity', 'eccentricity'), (0, 2, 3, 4, 5, 6), 3, 200]
[('area', 'circularity', 'eccentricity'), (1, 2, 3, 4, 6, 7), 11, 200]
[('area', 'perimeter', 'convexity', 'circularity'), (0, 1, 2, 4, 6, 7), 3, 100]
[('area', 'perimeter', 'rectangularity'), (0, 1, 3, 4, 5, 6), 7, 100]
[('area', 'perimeter', 'convexity', 'eccentricity'), (0, 1, 2, 3, 4, 5), 5, 50]


Forming the feature vectors both for train and test according to these combinations, and run the Nearest Neighbor

In [ ]:
for comb in rand_comb:

  print("Arbitrary Combination to be tested:", comb)

  for train_img in train_dict:

    vec = np.empty(0)

    basic = basic_shape_descriptors(train_img[1])
    vec = np.concatenate([vec, np.array([basic[el] for el in comb[0]])])

    moments = Hu_Moments(train_img[1])
    vec = np.concatenate([vec, np.array([moments[el] for el in comb[1]])])

    vec = np.concatenate([vec, Shape_Histogram(train_img[1], comb[2])])

    vec = np.concatenate([vec, Fourier_Descriptors(train_img[1], comb[3])])

    train_img[3] = vec.copy()

  for test_img in test_dict:

    vec = np.empty(0)

    basic = basic_shape_descriptors(test_img[1])
    vec = np.concatenate([vec, np.array([basic[el] for el in comb[0]])])

    moments = Hu_Moments(test_img[1])
    vec = np.concatenate([vec, np.array([moments[el] for el in comb[1]])])

    vec = np.concatenate([vec, Shape_Histogram(test_img[1], comb[2])])

    vec = np.concatenate([vec, Fourier_Descriptors(test_img[1], comb[3])])

    test_img[3] = vec.copy()

  Nearest_Neighbor(train_dict, test_dict)
  print("----------------------------------------------------------------------")



Arbitrary Combination to be tested: [('perimeter', 'convexity', 'eccentricity'), (0, 2, 3, 4, 5, 6), 3, 200]


<ipython-input-34-b76ca093f216>:2: RuntimeWarning: invalid value encountered in divide
  return np.sum((test_img - train_img) ** 2 / (test_img + train_img))
/usr/local/lib/python3.10/dist-packages/scipy/spatial/distance.py:1023: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(m)
<ipython-input-34-b76ca093f216>:2: RuntimeWarning: divide by zero encountered in divide
  return np.sum((test_img - train_img) ** 2 / (test_img + train_img))


Euclidean Score: 0.3414285714285714
Manhattan Score: 0.29428571428571426
Chi Squared Score: 0.37857142857142856
Mahalanobis Score: 0.008571428571428572
----------------------------------------------------------------------
Arbitrary Combination to be tested: [('area', 'circularity', 'eccentricity'), (1, 2, 3, 4, 6, 7), 11, 200]
Euclidean Score: 0.4642857142857143
Manhattan Score: 0.4614285714285714
Chi Squared Score: 0.3942857142857143
Mahalanobis Score: 0.01
----------------------------------------------------------------------
Arbitrary Combination to be tested: [('area', 'perimeter', 'convexity', 'circularity'), (0, 1, 2, 4, 6, 7), 3, 100]
Euclidean Score: 0.46714285714285714
Manhattan Score: 0.47
Chi Squared Score: 0.4942857142857143
Mahalanobis Score: 0.007142857142857143
----------------------------------------------------------------------
Arbitrary Combination to be tested: [('area', 'perimeter', 'rectangularity'), (0, 1, 3, 4, 5, 6), 7, 100]
Euclidean Score: 0.4671428571428571